# Horse Racing

# I. Introduction

Gambling on the horse racing results is a breathtaking entertainment. The results of horse races don’t come from nowhere. They are related to different factors such as the horses and jockeys, the track and distance, etc. We wonder whether we can “predict” the results of horse races and build our “formula” to win the rewards. In this project, we will use data from past races and try different machine learning techniques to make predictions.

The dataset is from kaggle (www.kaggle.com/lantanacamara/hong-kong-horse-racing) which is extracted from the website of the Hong Kongm Jockey Club. The file race-result-race.csv describes the races. Each entry in another file race-result-horse.csv corresponds to one horse in a race.

 # II.  Data Pre-processing

### Variable Description: 
• finishing_position: the rank of the horse. (E.g. the horse with finishing_position 1 is the first to finish)

• horse_number: the number for the horse in the specific race. (Note that the same horse may have different numbers in different races)

• horse_name: English name of the horse.

• horse_id: ID of the horse. (The ID for a horse is unique in all the races)

• jockey: the one who rides the horse in the race. (A jockey can ride different horses in the races)

• trainer: the one who trains the horse. (Multiple horses from a trainer can appear in the same race)

• actual_weight: the extra weight that the horse carries in the race. (The horses with better perfor- mances in the previous races should carry heavier extra weights, to make the race more competitive6)

• declared_horse_weight: the weight of the horse on date of the race.

• draw: the position of the horse at the starting point7. The inner positions are usually advantageous and correspond to smaller draw numbers.

• length_behind_winner: the length behind the winner at the finish line. The unit is the “horse length”.

• running_position_1: the rank of the horse at the first timing point.

• running_position_2: the rank of the horse at the second timing point.
...

• running_position_i: the rank of the horse at the ith timing point. (The running position will be “NA” if the total distance of the race is short and the horses don’t go across the particular timing point)

• finish_time: the total time from the starting point to the finish line. The unit is “second”.

• win_odds: the ratio between the reward you will get and the money you bet, supposing that you will win. The odds are usually determined automatically by the total money bet on each horses8. 

• race_id: the ID of the race for this entry. The race_id is consistent in the two data files.

In [64]:
import pandas as pd
import numpy as np

%load_ext autotime

import warnings
warnings.filterwarnings("ignore")

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2022-10-17 00:22:51 +08:00)


In [65]:
# Read the data
rrh = pd.read_csv('./data/race-result-horse.csv')
rrr = pd.read_csv('./data/race-result-race.csv')
print(rrh.shape)
print(rrr.shape)

(30189, 19)
(2367, 12)
time: 156 ms (started: 2022-10-17 00:22:52 +08:00)


In [66]:
# View the data
rrh.head()

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,1.0,2.0,2.0,1.0,1.22.33,3.8,NaN,NaN,2014-001
1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,8.0,9.0,9.0,2.0,1.22.65,8,NaN,NaN,2014-001
2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,2.0,1.0,1.0,3.0,1.22.66,5.7,NaN,NaN,2014-001
3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,6.0,4.0,5.0,4.0,1.22.66,6.1,NaN,NaN,2014-001
4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,9.0,10.0,10.0,5.0,1.23.02,6.1,NaN,NaN,2014-001


time: 16 ms (started: 2022-10-17 00:22:52 +08:00)


In [67]:
# View the data
rrr.head()

,src,race_date,race_course,race_number,race_id,race_class,race_distance,track_condition,race_name,track,sectional_time,incident_report
0,20140914-1.html,2014-09-14,Sha Tin,1,2014-001,Class 5,1400,GOOD TO FIRM,TIM WA HANDICAP,"TURF - ""A"" COURSE",13.59 22.08 23.11 23.55,\n When about to enter the trac...
1,20140914-10.html,2014-09-14,Sha Tin,10,2014-010,Class 2,1400,GOOD TO FIRM,COTTON TREE HANDICAP,"TURF - ""A"" COURSE",13.55 22.25 22.89 22.85,\n SMART MAN was slow to begin....
2,20140914-2.html,2014-09-14,Sha Tin,2,2014-002,Class 5,1200,GOOD TO FIRM,TIM MEI HANDICAP,"TURF - ""A"" COURSE",24.06 22.25 23.66,\n ALLEY-OOP and FLYING KEEPER ...
3,20140914-3.html,2014-09-14,Sha Tin,3,2014-003,Class 1,1200,GOOD TO FIRM,THE HKSAR CHIEF EXECUTIVE'S CUP (HANDICAP),"TURF - ""A"" COURSE",23.42 22.48 22.47,"\n On arrival at the Start, it ..."
4,20140914-4.html,2014-09-14,Sha Tin,4,2014-004,Class 4,1200,GOOD TO FIRM,LUNG WUI HANDICAP,"TURF - ""A"" COURSE",24.00 22.62 22.64,\n Just prior to the start bein...


time: 0 ns (started: 2022-10-17 00:22:52 +08:00)


In [68]:
# Explore the data
rrh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30189 entries, 0 to 30188
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   finishing_position     30187 non-null  object 
 1   horse_number           29851 non-null  float64
 2   horse_name             30189 non-null  object 
 3   horse_id               30189 non-null  object 
 4   jockey                 30189 non-null  object 
 5   trainer                30189 non-null  object 
 6   actual_weight          30189 non-null  object 
 7   declared_horse_weight  30189 non-null  object 
 8   draw                   30189 non-null  object 
 9   length_behind_winner   30189 non-null  object 
 10  running_position_1     29574 non-null  float64
 11  running_position_2     29560 non-null  float64
 12  running_position_3     29542 non-null  float64
 13  running_position_4     16618 non-null  float64
 14  finish_time            30189 non-null  object 
 15  wi

In [69]:
# Explore the data
rrr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2367 entries, 0 to 2366
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   src              2367 non-null   object
 1   race_date        2367 non-null   object
 2   race_course      2367 non-null   object
 3   race_number      2367 non-null   int64 
 4   race_id          2367 non-null   object
 5   race_class       2367 non-null   object
 6   race_distance    2367 non-null   int64 
 7   track_condition  2367 non-null   object
 8   race_name        2367 non-null   object
 9   track            2367 non-null   object
 10  sectional_time   2367 non-null   object
 11  incident_report  2367 non-null   object
dtypes: int64(2), object(10)
memory usage: 222.0+ KB
time: 0 ns (started: 2022-10-17 00:22:52 +08:00)


## Data Exploration

In [70]:
# Look at the unique values in the column 'finishing_position'
rrh['finishing_position'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', 'WV-A', '14', 'WV', '9 DH', '2 DH', 'WX', 'PU', '1 DH',
       '8 DH', 'UR', 'FE', '4 DH', '10 DH', 'TNP', '5 DH', 'DISQ',
       '11 DH', '12 DH', '3 DH', '7 DH', 'WX-A', 'DNF', '6 DH', nan],
      dtype=object)

time: 0 ns (started: 2022-10-17 00:22:53 +08:00)


In [71]:
# Drop rows in column where value is not a number
df_horse = rrh[rrh['finishing_position'].isin([str(i) for i in range(1, 15)])].reset_index(drop=True)

time: 16 ms (started: 2022-10-17 00:22:54 +08:00)


In [72]:
# View the data
df_horse.head()

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,1.0,2.0,2.0,1.0,1.22.33,3.8,NaN,NaN,2014-001
1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,8.0,9.0,9.0,2.0,1.22.65,8,NaN,NaN,2014-001
2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,2.0,1.0,1.0,3.0,1.22.66,5.7,NaN,NaN,2014-001
3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,6.0,4.0,5.0,4.0,1.22.66,6.1,NaN,NaN,2014-001
4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,9.0,10.0,10.0,5.0,1.23.02,6.1,NaN,NaN,2014-001


time: 0 ns (started: 2022-10-17 00:22:54 +08:00)


In [73]:
# Check the data types after removing special characters
df_horse.finishing_position.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14'], dtype=object)

time: 0 ns (started: 2022-10-17 00:22:55 +08:00)


In [74]:
df_horse

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,running_position_1,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id
0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,1.0,2.0,2.0,1.0,1.22.33,3.8,NaN,NaN,2014-001
1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,8.0,9.0,9.0,2.0,1.22.65,8,NaN,NaN,2014-001
2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,2.0,1.0,1.0,3.0,1.22.66,5.7,NaN,NaN,2014-001
3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,6.0,4.0,5.0,4.0,1.22.66,6.1,NaN,NaN,2014-001
4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,9.0,10.0,10.0,5.0,1.23.02,6.1,NaN,NaN,2014-001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29359,9,7.0,INTREPIC,A139,M F Poon,D J Hall,116,993,2,6,2.0,4.0,5.0,9.0,1.24.05,4,NaN,NaN,2016-805
29360,10,12.0,HIGH AND MIGHTY,S362,D Whyte,W Y So,119,1132,11,6-3/4,12.0,12.0,11.0,10.0,1.24.18,7.2,NaN,NaN,2016-805
29361,11,3.0,DOUBLE VALENTINE,A163,B Prebble,A S Cruz,127,1085,13,9-1/4,11.0,11.0,12.0,11.0,1.24.58,41,NaN,NaN,2016-805
29362,12,8.0,THE JOY OF GIVING,A249,W M Lai,C W Chang,122,1026,9,13-1/2,13.0,13.0,13.0,12.0,1.25.26,99,NaN,NaN,2016-805


time: 31 ms (started: 2022-10-17 00:22:58 +08:00)


In [75]:
# #  drop the rows where the “finish_position” is not a number (e.g. WV-A, WV). There may be accidents.
# rrh = rrh[rrh.finishing_position.isin(['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])].reset_index()
# rrh.head()

time: 16 ms (started: 2022-10-17 00:22:59 +08:00)


In [76]:
horse_id = df_horse['horse_id'].unique()
numHorse = len(horse_id)
horse_index = range(numHorse)

jockey = df_horse['jockey'].unique()
numJockey = len(jockey)
jockey_index = range(numJockey)

trainer = df_horse['trainer'].unique()
numTrainer = len(trainer)
trainer_index = range(numTrainer)

print(' Number of Horses:', numHorse, '\n', 'Number of Jockeys:', numJockey,'\n', 'Number of Trainers:', numTrainer)

 Number of Horses: 2155 
 Number of Jockeys: 105 
 Number of Trainers: 93
time: 0 ns (started: 2022-10-17 00:23:01 +08:00)


In [77]:
# Takes a while to run

# Add a column named recent_6_runs to the dataframe, which records the recent ranks of the horse in each entry. 
# The ranks are separated by “/”, and a record is like 1/2/6/3/4/7.

# Add a column named recent_ave_rank for each entry to the dataframe, 
# which records the average rank of the recent 6 runs of a horse

df_horse['recent_6_runs'] = '0'
df_horse['recent_ave_rank'] = '7'

for i in range(len(df_horse['finishing_position'])):
    
    temp = df_horse[ : (i + 1)][df_horse.horse_id == df_horse.horse_id[i]][['finishing_position']]
    temp = temp['finishing_position'].values.tolist()[::-1]
  
    df_horse['recent_6_runs'][i] = '/'.join(temp[:6])
    
    if len(temp) != 0:
        temp_int = map(int,temp)    # convert string to integer
        temp_ave = np.mean(list(temp_int))
        df_horse['recent_ave_rank'][i] = temp_ave

time: 2min 38s (started: 2022-10-17 00:23:13 +08:00)


In [78]:
# Check the data
df_horse.tail()

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,...,running_position_2,running_position_3,running_position_4,finish_time,win_odds,running_position_5,running_position_6,race_id,recent_6_runs,recent_ave_rank
29359,9,7.0,INTREPIC,A139,M F Poon,D J Hall,116,993,2,6,...,4.0,5.0,9.0,1.24.05,4,NaN,NaN,2016-805,9/1/2/12/13/13,8.75
29360,10,12.0,HIGH AND MIGHTY,S362,D Whyte,W Y So,119,1132,11,6-3/4,...,12.0,11.0,10.0,1.24.18,7.2,NaN,NaN,2016-805,10/5/3/9/8/9,6.333333
29361,11,3.0,DOUBLE VALENTINE,A163,B Prebble,A S Cruz,127,1085,13,9-1/4,...,11.0,12.0,11.0,1.24.58,41,NaN,NaN,2016-805,11/8/11,10.0
29362,12,8.0,THE JOY OF GIVING,A249,W M Lai,C W Chang,122,1026,9,13-1/2,...,13.0,13.0,12.0,1.25.26,99,NaN,NaN,2016-805,12/12/10,11.333333
29363,13,5.0,MIGHTY BOY,A352,N Callan,J Moore,126,1153,1,14-1/4,...,5.0,4.0,13.0,1.25.35,41,NaN,NaN,2016-805,13,13.0


time: 15 ms (started: 2022-10-17 00:25:52 +08:00)


In [79]:
# Add column of Distance

# The distance could be 1000, 1200, 1400, 1600, 1800, 2000, 2400, etc. 
# Some horses are good at short-distance races, while some are good at long-distance races.

df_horse['race_distance'] = '0'

# Iterate through the dataframe
for i in range(len(rrr['race_id'])):
    distance = rrr['race_distance'][i]
    df_horse['race_distance'][df_horse.race_id == rrr.race_id[i]] = distance

time: 6.06 s (started: 2022-10-17 00:25:52 +08:00)


In [82]:
# Add HorseWin, HorseRankTop3, HorseRankTop50Percent for use in Part3

# Sets a boolean mask (either 1 or 0) for each new column
df_horse['HorseWin'] = (df_horse.finishing_position == '1') + 0
df_horse['HorseRankTop3'] = (df_horse.finishing_position == '1') + (df_horse.finishing_position == '2') + (df_horse.finishing_position == '3') + 0

# Create a list of all the top1_indexes where the horse finished in the top
top1_index = df_horse.index[df_horse['finishing_position'] == '1'].tolist()

# Create another column where the horse finished in the top 50%
df_horse['HorseRankTop50Percent'] = '0'
for i in range(len(top1_index)-1):
    df_horse['HorseRankTop50Percent'][top1_index[i]:(top1_index[i] + int(round(0.5 * (top1_index[i + 1] - top1_index[i]))))] = '1'

# Created specifically for the last index because we cannot loop    
df_horse['HorseRankTop50Percent'][top1_index[len(top1_index) - 1]:(top1_index[len(top1_index) - 1] + 6)] = '1'


time: 937 ms (started: 2022-10-17 00:27:43 +08:00)


In [83]:
df_horse.head()

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,...,win_odds,running_position_5,running_position_6,race_id,recent_6_runs,recent_ave_rank,race_distance,HorseWin,HorseRankTop3,HorseRankTop50Percent
0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,...,3.8,NaN,NaN,2014-001,1,1.0,1400,1,1,1
1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,...,8,NaN,NaN,2014-001,2,2.0,1400,0,1,1
2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,...,5.7,NaN,NaN,2014-001,3,3.0,1400,0,1,1
3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,...,6.1,NaN,NaN,2014-001,4,4.0,1400,0,0,1
4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,...,6.1,NaN,NaN,2014-001,5,5.0,1400,0,0,1


time: 16 ms (started: 2022-10-17 00:27:44 +08:00)


In [84]:
# Save the data
df_horse.to_csv('./data/df_horse.csv', index=False)

time: 204 ms (started: 2022-10-17 00:28:17 +08:00)


## Data Preparation

In [86]:
# Divide data into train set and test set.
# Training set contains all the races with race_id no more than “2016-327” (first 80%), 
# Test set contains the rest of races (last 20%)

temp_index = max(rrh.index[rrh['race_id'] == '2016-327'].tolist())
df_train = df_horse[:(temp_index + 1)]
df_test = df_horse[(temp_index + 1):]


time: 0 ns (started: 2022-10-17 00:44:09 +08:00)


In [87]:
# Calculate jockey_ave_rank, trainer_ave_rank ONLY for training data
# jockey_ave_rank: records the average rank of the jockey in the training data
# trainer_ave_rank: records the average rank of the trainer in the training data

df_train['jockey_ave_rank'] = '7'

for i in range(len(jockey)):
    temp = df_train[df_train.jockey == jockey[i]][['finishing_position']]
    temp = temp['finishing_position'].values.tolist()
    if len(temp) != 0:
        temp_int = map(int, temp)
        temp_ave = np.mean(list(temp_int))
        df_train['jockey_ave_rank'][df_train.jockey == jockey[i]] = temp_ave


df_train['trainer_ave_rank'] = '7'

for i in range(len(trainer)):
    temp = df_train[df_train.trainer == trainer[i]][['finishing_position']]
    temp = temp['finishing_position'].values.tolist()
    if len(temp) != 0:
        temp_int = map(int,temp)
        temp_ave = np.mean(list(temp_int))
        df_train['trainer_ave_rank'][df_train.trainer == trainer[i]] = temp_ave


time: 875 ms (started: 2022-10-17 00:49:45 +08:00)


In [14]:
# Save the train set
df_train.to_csv('df_train.csv')

In [88]:
# Port over the jockey_ave_rank and trainer_ave_rank to the test set

df_test['jockey_ave_rank'] = '7'
df_test_jockey = df_test['jockey'].unique()
for i in range(len(df_test_jockey)):
    if df_test_jockey[i] in df_train['jockey'].unique():
        df_test['jockey_ave_rank'][df_test.jockey == df_test_jockey[i]]\
            = df_train[df_train.jockey == df_test_jockey[i]]['jockey_ave_rank'].tolist()[0]

df_test['trainer_ave_rank'] = '7'
df_test_trainer=df_test['trainer'].unique()
for i in range(len(df_test_trainer)):
    if df_test_trainer[i] in df_train['trainer'].unique():
        df_test['trainer_ave_rank'][df_test.trainer == df_test_trainer[i]]\
            = df_train[df_train.trainer == df_test_trainer[i]]['trainer_ave_rank'].tolist()[0]

time: 281 ms (started: 2022-10-17 00:51:42 +08:00)


In [89]:
df_test.to_csv('test.csv')

time: 63 ms (started: 2022-10-17 00:51:50 +08:00)
